# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [ ]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [20]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['shipping', 'payments', 'reviews'],
        'definition': ['Shipping details: shipping method, cost, and delivery time',
                       'Payment details: payment method, amount paid, and payment status',
                       'Product reviews: customer rating, comments, and review date']}
df = pd.DataFrame(data)
print(df)

      table                                         definition
0  shipping  Shipping details: shipping method, cost, and d...
1  payments  Payment details: payment method, amount paid, ...
2   reviews  Product reviews: customer rating, comments, an...


In [21]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [22]:
print(text_tables)

shipping: Shipping details: shipping method, cost, and delivery time
payments: Payment details: payment method, amount paid, and payment status
reviews: Product reviews: customer rating, comments, and review date


In [23]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [24]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="What is the average shipping cost for orders delivered via express method?")

In [25]:
print(return_OAI(pqt1))

```json
{
    "tables": ["shipping"]
}
```


In [26]:
pqt3 =prompt_question_tables.format(tables=text_tables,
                                     question="How many reviews did each product receive, and what is the average rating per product?")

In [27]:
print(return_OAI(pqt3))

{
    "tables": ["reviews"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [28]:
pqt4 = prompt_question_tables.format(tables=text_tables,
                                     question="What payment methods were used the most, and how much total was paid using each method?")
print(return_OAI(pqt4))

```json
{
    "tables": ["payments"]
}
```


In [29]:
pqt5 = prompt_question_tables.format(tables=text_tables,
                                     question="Which products received the highest ratings and what were the most common comments associated with them?")
print(return_OAI(pqt5))

```json
{
    "tables": ["reviews"]
}
```


Summary Report: SQL Query from Table Names Using GPT-3.5-Turbo
Objective:
This experiment aimed to assess whether GPT-3.5-Turbo can accurately identify the relevant tables required for SQL queries using only table names and brief descriptions. The approach involved structured prompts, feeding the model with tabular metadata, and evaluating its responses for accuracy.

Findings:
Effective Scenarios:
 Accurate Table Selection: When table names and descriptions clearly aligned with the query, GPT-3.5-Turbo successfully identified the necessary tables.
Structured Prompts Yield Better Results: Well-defined instructions and formatting significantly improved accuracy and consistency.
Handling Multi-Table Queries: The model correctly inferred relevant tables when handling queries like determining the fastest shipping method with the lowest cost—identifying the shipping table as essential.
Inference in Ambiguous Cases: When the query was vague (e.g., identifying necessary tables without explicit conditions), the model still inferred key relationships between tables like payments, reviews, and shipping to generate reasonable responses.

Challenging Scenarios:
 Hallucinations: In some cases, the model included irrelevant tables that were not needed for the specific query.
 Over-Inclusion of Tables: The model occasionally overcompensated by selecting more tables than necessary (e.g., including the reviews table when a query was solely about shipping).
 Table Omission in Edge Cases: For certain complex queries involving filtering (e.g., finding the best-rated shipping provider based on past reviews), the model sometimes omitted relevant tables.
 Prompt Sensitivity: Small changes in how the query was phrased affected consistency, sometimes leading to different sets of selected tables for similar questions.

Key Learnings:
 Prompt Optimization is Crucial: The quality of the prompt directly impacts the model’s ability to identify correct tables.
 Explicit Table Descriptions Improve Accuracy: Vague descriptions result in hallucinations or omissions—clearer metadata helps guide the model.
 Iterative Testing is Beneficial: Running multiple variations of the same query can help refine the prompt structure for better results.
 Explicit Table Relationships Help: Clearly stating how tables are linked in the database schema improves accuracy in multi-table queries.
 GPT-3.5-Turbo's Relational Reasoning is Limited: While it can infer relationships based on descriptions, it does not inherently understand SQL schema dependencies.

Future Considerations:
 Improve Table Definitions: Adding more context-rich, detailed descriptions will enhance accuracy.
 Provide Contextual Examples: Including sample queries with expected table selections can improve the model’s ability to generalize correctly.
 Manually Validate Outputs: Cross-checking model predictions against actual database schemas can help detect hallucinations.
 Fine-Tune for SQL Tasks: Training a specialized model for database-related tasks could improve precision.
 Explicitly Prioritize Relationships: Designing prompts that highlight inter-table relationships will aid in handling complex queries.

